In [1]:
import h5py

import numpy as np
import pandas as pd

from glob import glob

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Age and 4 anonymized targets, 443 partially missed observations
train_scores = pd.read_csv('../input/train_scores.csv').sort_values(by='Id')

# Somehow preprocessed morphometry (after group ICA), simplest feature set
loadings = pd.read_csv('../input/loading.csv')

# resting-state fMRI Functional Network Connectivity matrices. 
# In simple setting, these are cross-correlations (in this case something more sophisticated) between
# every pair of brain regions presented in train/test *.mat
fnc = pd.read_csv('../input/fnc.csv')

# Submit Age and 4 scores for test ids
sample = pd.read_csv('../input/sample_submission.csv')

# List of some of subjects from test set whose data were collected from different scanner
reveal = pd.read_csv('../input/reveal_ID_site2.csv')

# 53 unique numbers between 2 and 99 (somehow related to brain regions? regions keys?)
icn_nums = pd.read_csv('../input/ICN_numbers.csv')


In [25]:
degree = pd.read_csv('../input/degree.csv')

In [26]:
degree_columns = degree.columns[:-1]

degree[degree_columns] /= 10000

In [28]:
fnc = pd.read_csv('../input/fnc.csv')
fnc_columns = fnc.columns[1:]
fnc[fnc_columns] /= 500

In [29]:
fnc.iloc[:, 1:].max().max()

0.00198444108

In [30]:
fnc.iloc[:, 1:].min().min()

-0.0019217890600000001

In [31]:
degree.iloc[:, :-1].max().max()

0.002191081879

In [32]:
degree.iloc[:, :-1].min().min()

-0.002281362082

In [33]:
degree.shape

(11754, 54)

In [34]:
data = pd.merge(degree, train_scores, on='Id').dropna()

In [37]:
data[fe]

,degree_2,degree_3,degree_4,degree_5,degree_7,degree_8,degree_9,degree_11,degree_12,degree_13,...,degree_94,degree_96,degree_98,degree_99,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2
0,0.000070,-0.000046,-0.000217,-0.000118,-0.000197,-0.000091,0.000096,0.000004,-0.000048,0.000125,...,-0.000186,-0.000152,-0.000071,-0.000038,10001,57.436077,30.571975,62.553736,53.325130,51.427998
1,-0.000257,-0.000009,-0.000175,0.000199,0.000084,0.000134,-0.000072,-0.000143,0.000240,0.000062,...,-0.000347,-0.000320,0.000057,0.000134,10002,59.580851,50.969456,67.470628,60.651856,58.311361
2,-0.000057,0.000065,-0.000122,-0.000023,0.000014,0.000042,0.000049,0.000059,0.000034,-0.000015,...,-0.000187,-0.000196,-0.000017,0.000028,10004,71.413018,53.152498,58.012103,52.418389,62.536641
4,-0.000331,-0.000374,-0.000012,0.000049,0.000066,0.000109,-0.000340,-0.000405,0.000026,0.000131,...,-0.000157,0.000045,0.000088,0.000028,10007,38.617381,49.197021,65.674285,40.151376,34.096421
5,-0.000174,-0.000104,-0.000090,-0.000171,-0.000049,-0.000150,-0.000080,-0.000061,-0.000141,-0.000064,...,-0.000123,-0.000083,-0.000118,-0.000008,10008,35.326582,15.769168,65.782269,44.643805,50.448485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5872,-0.000048,-0.000064,-0.000045,-0.000143,-0.000180,-0.000113,-0.000091,-0.000039,-0.000017,-0.000011,...,-0.000130,-0.000135,-0.000003,0.000016,21746,14.257265,21.358872,61.165998,51.778483,54.640179
5873,-0.000101,-0.000044,-0.000148,-0.000073,-0.000072,-0.000194,-0.000014,-0.000075,-0.000073,-0.000102,...,-0.000029,0.000009,-0.000114,-0.000151,21747,55.456978,68.169675,29.907995,55.349257,54.019517
5874,-0.000151,-0.000253,-0.000120,-0.000117,-0.000152,-0.000071,-0.000201,-0.000251,-0.000012,0.000047,...,-0.000099,-0.000140,-0.000111,0.000064,21750,48.948756,55.114811,60.878271,38.617246,50.679885
5875,-0.000197,-0.000074,0.000014,-0.000008,-0.000005,0.000059,-0.000200,-0.000095,-0.000279,0.000103,...,0.000052,0.000067,0.000080,0.000145,21752,66.532630,59.844808,72.303110,55.458281,46.870235


In [ ]:
deg_train = np.asarray(data.drop('Id', axis=1))

In [11]:
import re
from tqdm import tqdm

In [12]:
idxs = []
r = re.compile('\d+')
col_dict = {}
for col in fnc.columns:
    ind = r.findall(col)
    if ind:
        col_dict[col] = [int(i) for i in ind]
        idxs += ind

In [13]:
matrix = np.zeros((100, 100))

In [14]:
for row in tqdm(fnc.iterrows()):
    break

0it [00:00, ?it/s]


In [15]:
df_row = row[1]

In [16]:
df_row

Id                    10001.000000
SCN(53)_vs_SCN(69)        0.368580
SCN(98)_vs_SCN(69)        0.166876
SCN(99)_vs_SCN(69)        0.438148
SCN(45)_vs_SCN(69)        0.341007
                          ...     
CBN(4)_vs_CBN(13)        -0.042853
CBN(7)_vs_CBN(13)         0.124627
CBN(4)_vs_CBN(18)        -0.060712
CBN(7)_vs_CBN(18)         0.515964
CBN(7)_vs_CBN(4)          0.290488
Name: 0, Length: 1379, dtype: float64

In [17]:
i, j = col_dict[col]

In [18]:
i, j

(7, 4)

In [19]:
matrix[i, j] = df_row[col]

In [20]:
matrix += matrix.T

In [21]:
def get_matrix(df_row, return_idx=False):
    matrix = np.zeros((100, 100))
    for col in df_row.index[1:]:
        i, j = col_dict[col]
        matrix[i, j] = df_row[col]
    matrix += matrix.T
    
    idx = np.array([ 2,  3,  4,  5,  7,  8,  9, 11, 12, 13, 15, 16, 17, 18, 20, 21, 23,
                     27, 32, 33, 37, 38, 40, 43, 45, 48, 51, 53, 54, 55, 56, 61, 62, 63,
                     66, 67, 68, 69, 70, 71, 72, 77, 79, 80, 81, 83, 84, 88, 93, 94, 96,
                     98, 99])
    if return_idx:
        return matrix[:, idx][idx, :], idx 
    return matrix[:, idx][idx, :]

In [22]:
degrees = []
for row in tqdm(fnc.iterrows()):
    mat = get_matrix(row[1])
    degrees.append(mat.sum(axis=1))

11754it [02:50, 68.95it/s]


In [23]:
_, idx = get_matrix(fnc.iloc[0], return_idx=True)
degrees = pd.DataFrame(degrees, columns=idx)
degrees['Id'] = fnc['Id']

In [30]:
degrees_columns = degrees.columns[:-1]

In [32]:
degrees.columns = [f'degree_{f}' for f in degrees_columns] + ['Id']

In [34]:
degrees.to_csv('../input/degree.csv', index=False)